*Glory to Jesus Christ!*
# Financial Calculus: An introduction to derivative pricing.
## Chapter 1: Introduction

### The Parable of the Bookmaker

A bookmaker takes bets on a two-horse race.
A very sharp bookmaker, he correctly calculates that Horse 1's chance of winning is 25%, and Horse 2's chance of winning is 75%.
Hence, the odds are set at 3-1 against Horse 1, and 3-1 on Horse 2.
But, popular sentiment does not reflect the bookmaker's calculations: 5000\\$ are bet on Horse 1, and 10000\\$ on Horse 2.

If Horse 2 were to win, the bookmaker would make a net profit of $5000 - (1/3)*10000 = 1667\$ $.
But, if Horse 1 were to win, the bookmaker would make a net profit of $10000 - (3/1)*5000 = -5000\$ $. The expected value of his profit is $25%*(-5000) + 75%*(1667) \approx 0\\$ $, or exactly even. In the long term, over a number of similar but independent races, the bookmaker would break even. Until then, there is a chance of making a large loss.

Instead, suppose the bookmaker had set odds *according to the wagered money*: that is, 2-1 against and 2-1 on respectively. Our new outcomes are:
$5000 - (1/2)*10000 = 0\$ $ should Horse 2 win, and $10000 - (2/1) * 5000 = 0\$ $ should Horse 1 win.

The bookmaker breaks even every time. The outcome is irrelevant. In practice, the bookmaker sells more than 100% of the race, and shortens the odds for profit. However, the same pattern emerges: for the bookmaker to extract a steady riskless income, he assumes the horse's chances from the amounts of money wagered, and not from what they are in reality.

This puts the bookmaker in the surprising position of being uninterested in the outcome of the race. His profit is assured.

In [ ]:
# Assumes a two-horse race. Use positive numbers.
function corrected_odds(h1, 

### Expectation Pricing

Take for example a coin-flip game. A fair coin is flipped, paying you one dollar for heads and nothing for tails.

Profit follows expectation, which follows probability. In discrete cases, the expectation is the total of each outcome's value weighted by its attached probability. The probability of the coin landing heads or tails is $\frac{1}{2}$. Hence, the expected payoff in the game is:
$$ \frac{1}{2} * 1\$ + \frac{1}{2} * 0\$ = 0.50 \$ $$
We link this formal expectation to a 'price' for the game via something like the **Kolmogorov strong law of large numbers**.

>#### Kolmogorov's Strong Law of Large Numbers:
>Suppose there exists a sequence of independent random numbers $X_1, \ldots, X_n$ all sampled from the same distribution with mean $\mu$. We let $S_n$ be the arithmetical average of the sequence up to the $n$th term, i.e.
$$S_n = (X_1 + \ldots + X_n)/n$$
>With probability 1, as $n$ gets larger the value of $S_n$ approaches the mean $\mu$ of the distribution.

The average profit (or loss) per game tends towards the mathematical expectation, less the price paid to play the game. If this difference is positive, then the game is profitable over the long-run (with certainty). If the difference is negative, the player will approach a guaranteed loss with certainty. In this sense, fifty cents is a fair price for the coin-game.

But, in the *short-term*, nothing is guaranteed.